In [9]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.2 MB/s eta 0:00:00


In [20]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config
from transformers import get_linear_schedule_with_warmup

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split, RandomSampler, SequentialSampler

import pandas as pd

In [ ]:
def load_prompts(file_path):
    """
    Reads a text file where harmful prompts start with "-" and may be separated by "\n" within a line.
    """
    prompts = []
    with open(file_path, "r") as f:
        for line in f:
            s = f"<|startoftext|>{line} <|endoftext|>"
            prompts.append(s)
    return prompts

In [ ]:
data = []
for i in [1,2,3,4,5,6,7,8,9]:
  file_path = f'/content/drive/MyDrive/Colab Notebooks/data_gen8/results_{i}.txt'
  data += load_prompts(file_path)

In [ ]:
print(data[10])

<|startoftext|> 
 <|endoftext|>


In [ ]:
model_name = "gpt2-medium"

tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )

In [ ]:
batch_size = 5
max_length = 180

# standard PyTorch approach of loading data in using a Dataset class.
class HarmfulPromptsDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for dt in data:
            encodings = tokenizer.encode_plus(dt,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=max_length,
                                              return_tensors='pt'  # return a PyTorch tensor
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))


    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

#dataset = HarmfulPromptsDataset(data[:1000], tokenizer)
dataset = HarmfulPromptsDataset(data, tokenizer)
print(f"input_ids: {dataset[0][0]} attn_masks: {dataset[0][1]}")

input_ids: tensor([50257,   220,   198,   220, 50256, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 502

In [ ]:
# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create the DataLoaders for our training and validation datasets.
# Get training samples in random order.
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size # Trains with this batch size.
        )

# Get valiation samples sequentially.
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size # Evaluate with this batch size.
        )

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-medium"
model_save_path = '/content/drive/MyDrive/Colab Notebooks/model_gen6/'


In [ ]:
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_save_path)
model.to(device)


epochs = 3
learning_rate = 2e-5
warmup_steps = 1e2
# to prevent any division by zero in the implementation
epsilon = 1e-8
optim = AdamW(model.parameters(), lr = learning_rate, eps = epsilon)

total_steps = len(train_dataloader) * epochs  # [no batches] x [no epochs]

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
def infer(prompt):
    input = f"<|startoftext|>Prompt: {prompt.strip()}. <|endoftext|>"
    input = tokenizer(input, return_tensors="pt")
    input_ids      = input["input_ids"]
    attention_mask = input["attention_mask"]

    output = model.generate(input_ids.to(device),
                            attention_mask=attention_mask.to(device),
                            max_new_tokens=max_length,
                            do_sample = True, top_k = 50, top_p = 0.95, temperature = 1)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output


In [ ]:
for epoch_i in range(0, epochs):
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_labels    = batch[0].to(device)
        b_masks     = batch[1].to(device)

        model.zero_grad()
        outputs = model(input_ids = b_input_ids, labels = b_labels,
                         attention_mask = b_masks, token_type_ids = None )

        loss = outputs[0]

        # Get sample every x batches.
        if step % 100 == 0 and not step == 0:
            model.eval()
            input_sequence = "<|startoftext|>"
            inputs = torch.tensor(tokenizer.encode(input_sequence)).unsqueeze(0)

            output = model.generate(inputs.to(device),
                                    do_sample=True,
                                            top_k=50,
                                            max_length = 300,
                                            top_p=0.95,
                                            num_return_sequences=1)


            output_text = tokenizer.decode(output[0], skip_special_tokens=True)
            print(output_text)
            model.train()

        loss.backward()
        optim.step()
        scheduler.step()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
model.eval()
# Evaluate data for one epoch
for batch in validation_dataloader:
  b_input_ids = batch[0].to(device)
  b_labels    = batch[0].to(device)
  b_masks     = batch[1].to(device)

  with torch.no_grad():
    outputs  = model(input_ids = b_input_ids, labels = b_labels, attention_mask = b_masks)
    loss = outputs[0]

In [ ]:
model_gen_save_path = '/content/drive/MyDrive/Colab Notebooks/model_gen7/'
model.save_pretrained(model_gen_save_path)
tokenizer.save_pretrained(model_gen_save_path)

('/content/drive/MyDrive/Colab Notebooks/model_gen7/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/model_gen7/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/model_gen7/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/model_gen7/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/model_gen7/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/model_gen7/tokenizer.json')

In [ ]:
def generate_sequences(top_k = 50, top_p=0.85, temperature = 0.7):
  input_sequence = "<|startoftext|>"
  inputs = torch.tensor(tokenizer.encode(input_sequence)).unsqueeze(0)

  output = model.generate(inputs.to(device),
                          do_sample=True,
                                  top_k=top_k,
                                  max_length = 300,
                                  top_p=top_p,
                                  temperature = temperature,
                                  num_return_sequences=100)


  output_text = tokenizer.decode(output[0], skip_special_tokens=True)
  print(output_text)
  return output